TVAE Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `TVAE` model, including how to:

-   Create an instance of `TVAE`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `TVAE` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is TVAE?
--------------

The `sdv.tabular.TVAE` model is based on the VAE-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `TVAE` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `tvae` installed on your
system. If you have not done it yet, please install `tvae` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


In [2]:
from sdv.tabular import TVAE

model = TVAE()
model.fit(data)

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying TVAESynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17378,M,56.118267,51.133251,Commerce,64.391594,Comm&Mgmt,True,0,66.217881,Mkt&Fin,63.389286,33135.508049,True,2020-01-07,NaT,12.0
1,17360,M,52.399152,69.713852,Commerce,66.484079,Sci&Tech,True,0,71.139885,Mkt&Fin,60.030021,25135.727884,True,2020-01-12,NaT,NaN
2,17401,M,56.769211,76.223920,Commerce,63.120233,Comm&Mgmt,True,0,68.509504,Mkt&HR,59.189223,33697.856191,True,2019-12-30,2020-09-18,12.0
3,17457,M,60.090572,67.569741,Arts,71.503238,Comm&Mgmt,True,1,73.017871,Mkt&HR,62.918892,NaN,True,2020-01-10,2020-04-28,12.0
4,17386,M,61.792445,64.112261,Commerce,72.382221,Sci&Tech,True,0,69.386004,Mkt&HR,62.341545,31246.703896,True,2020-01-06,NaT,NaN


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be sent over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `TVAE.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = TVAE.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `tvae` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking at the demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17399,M,60.678583,79.970440,Arts,73.859212,Comm&Mgmt,True,1,60.350648,Mkt&HR,61.365319,NaN,False,2020-01-09,NaT,12.0
15,17399,M,63.823198,59.190034,Commerce,62.232075,Sci&Tech,True,1,58.237778,Mkt&HR,62.138536,31537.935153,True,2020-01-11,2020-10-08,12.0
73,17399,M,54.287164,67.975751,Commerce,73.383891,Comm&Mgmt,True,0,71.407629,Mkt&HR,60.138541,41387.402022,False,2020-01-07,NaT,12.0
85,17399,M,66.318493,66.086744,Science,72.060351,Comm&Mgmt,True,0,72.986934,Mkt&HR,59.187607,30835.716229,True,2020-01-05,NaT,12.0
89,17399,M,62.398861,68.306493,Commerce,72.548003,Comm&Mgmt,True,0,57.271179,Mkt&HR,62.142534,38557.870565,True,2020-01-07,NaT,NaN
111,17399,M,63.466775,64.372279,Commerce,64.659928,Comm&Mgmt,True,0,59.790013,Mkt&HR,64.195586,NaN,True,NaT,NaT,NaN
173,17399,M,65.644201,85.704198,Commerce,62.144493,Comm&Mgmt,True,0,73.248281,Mkt&Fin,61.623972,40714.624040,True,2020-03-02,NaT,12.0
182,17399,M,58.598760,57.099695,Commerce,60.437257,Comm&Mgmt,True,1,56.882455,Mkt&Fin,59.704108,36964.352125,True,NaT,NaT,NaN


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = TVAE(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,74.401652,68.506060,Science,71.094650,Comm&Mgmt,False,1,90.549180,Mkt&HR,56.521856,22709.003113,True,NaT,NaT,NaN
1,1,F,58.890587,63.042865,Science,64.571510,Comm&Mgmt,False,0,80.519106,Mkt&HR,66.818646,60266.903939,True,NaT,NaT,6.0
2,2,F,75.867496,61.393908,Science,64.051598,Comm&Mgmt,True,1,67.854064,Mkt&Fin,60.262410,26111.058341,False,2020-01-10,NaT,6.0
3,3,F,64.075336,63.691586,Science,78.592005,Comm&Mgmt,False,1,90.511921,Mkt&HR,66.019954,25646.258253,True,2020-01-27,NaT,NaN
4,4,F,74.629813,65.412998,Science,61.858252,Comm&Mgmt,True,0,62.299872,Mkt&HR,58.633474,34448.769488,False,NaT,NaT,6.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = TVAE(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"752 Johnson Turnpike\nWrightfurt, NY 74928",F,76.758485,76.112681,Arts,65.959233,Others,True,0,71.091833,Mkt&Fin,56.260665,48242.138547,True,NaT,NaT,12.0
1,1,Unit 0879 Box 3878\nDPO AP 42663,F,61.438246,60.240227,Science,72.862650,Others,False,0,83.836413,Mkt&HR,66.836261,NaN,True,NaT,NaT,6.0
2,2,"0917 Shawn Grove Suite 337\nWhiteville, KS 92476",F,62.098157,60.770410,Arts,64.730022,Others,False,2,57.507839,Mkt&Fin,64.225982,NaN,False,NaT,NaT,12.0
3,3,"752 Johnson Turnpike\nWrightfurt, NY 74928",F,77.093460,63.973584,Science,64.605227,Others,True,0,72.406209,Mkt&HR,62.740907,39162.183658,True,NaT,2020-08-06,6.0
4,4,Unit 0879 Box 3878\nDPO AP 42663,F,77.638767,73.235173,Arts,66.087621,Others,True,0,56.919886,Mkt&HR,60.580407,35809.926481,True,NaT,NaT,6.0


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = TVAE(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"553 Kristin Circle Apt. 058\nMontgomeryberg, O...",M,72.965851,71.992283,Arts,62.374113,Others,False,1,61.991021,Mkt&Fin,57.826065,NaN,False,2020-07-07,2020-04-21,3.0
1,1,"20825 Montes Pines\nStacyshire, TN 45091",F,76.616330,68.451534,Science,71.373953,Comm&Mgmt,True,1,61.643536,Mkt&Fin,56.993880,NaN,False,2020-03-01,2020-09-14,12.0
2,2,"9860 Henderson Lane Apt. 925\nGabrielberg, IL ...",F,77.449125,51.435321,Arts,69.517086,Others,False,2,74.750410,Mkt&Fin,56.442619,NaN,False,2020-03-05,2020-10-31,NaN
3,3,"72862 Shane Island\nRichardside, WV 33846",F,73.323144,67.039100,Arts,67.886453,Sci&Tech,True,2,60.627766,Mkt&Fin,60.338809,NaN,False,2020-03-03,NaT,NaN
4,4,"561 Christopher Drives Suite 962\nNew Lisa, UT...",M,74.680793,49.976534,Arts,67.013806,Sci&Tech,True,1,65.592413,Mkt&Fin,56.470211,NaN,False,2020-03-12,NaT,12.0


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.

### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the `TVAE` model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [17]:
conditions = {
    'gender': 'M'
}
model.sample(5, conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,"252 Danielle Burgs Apt. 514\nJohnsonberg, WY 3...",M,78.367132,56.344973,Arts,73.552056,Others,True,1,59.314468,Mkt&Fin,57.045508,NaN,False,2020-03-07,2020-11-11,NaN
1,2,"6065 Ramirez Island\nNathanielfort, CT 34402",M,73.359565,65.776466,Arts,71.738285,Sci&Tech,False,1,58.694949,Mkt&Fin,56.724116,NaN,False,2020-03-01,2020-05-08,12.0
2,4,"8014 Michele Turnpike\nLake Kathy, MT 96697",M,77.921706,68.110784,Arts,68.054646,Sci&Tech,False,2,58.982682,Mkt&Fin,63.662012,NaN,False,2020-03-12,2020-04-23,NaN
3,1,"573 David Lane\nStevensfurt, CT 26716",M,80.016361,62.529507,Arts,64.871461,Others,False,1,86.094837,Mkt&Fin,55.958669,NaN,True,2020-07-18,NaT,12.0
4,2,"5287 Rhonda Cove Apt. 526\nKellyview, VA 75774",M,77.415685,65.720098,Arts,67.305967,Comm&Mgmt,False,2,64.979630,Mkt&Fin,54.685492,NaN,False,2020-03-09,NaT,NaN


It's also possible to condition on multiple columns, such as `gender = M, 'experience_years': 0`.

In [18]:
conditions = {
    'gender': 'M',
    'experience_years': 0
}
model.sample(5, conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,"906 Pacheco Corners Apt. 492\nAndrealand, MA 2...",M,77.141555,69.891159,Science,69.717555,Comm&Mgmt,True,0,55.180724,Mkt&Fin,55.601112,NaN,False,2020-03-04,2020-08-11,12.0
1,8,"0180 Richard Cape\nSouth Matthewmouth, SD 09780",M,72.705238,63.708279,Arts,70.490066,Comm&Mgmt,True,0,89.410588,Mkt&HR,57.276541,NaN,False,2020-03-10,NaT,NaN
2,13,"666 Corey Square Apt. 203\nLisabury, KY 22380",M,74.573001,67.269364,Arts,71.389064,Sci&Tech,True,0,57.774791,Mkt&Fin,57.571041,NaN,False,2020-03-06,NaT,NaN
3,16,"799 Michelle Plains\nFordview, GA 69202",M,81.702955,59.503841,Arts,66.079633,Others,True,0,58.502401,Mkt&Fin,55.732981,NaN,False,2020-03-10,NaT,12.0
4,26,73822 Jeffrey Ridge Apt. 076\nPort Melissaview...,M,80.165317,69.003897,Science,79.898090,Others,True,0,73.735787,Mkt&Fin,57.826902,NaN,False,2020-03-07,NaT,12.0


`conditions` can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`, we can do the following: 

In [19]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
})
model.sample(conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,2,"854 Rickey Plains\nKimport, HI 57341",M,76.035707,65.720531,Arts,64.113725,Sci&Tech,True,1,61.501540,Mkt&Fin,60.366774,NaN,False,2020-03-12,NaT,3.0
1,0,"083 Michael Island\nNorth Grantville, CO 14773",M,72.182756,50.058279,Arts,72.554990,Comm&Mgmt,True,1,69.285402,Mkt&Fin,57.141104,NaN,False,2020-01-18,NaT,NaN
2,1,"706 Allison Lane Suite 902\nHerreramouth, OR 1...",M,70.598199,49.755725,Arts,72.058614,Comm&Mgmt,True,0,67.801302,Mkt&Fin,57.561687,NaN,False,2020-03-06,NaT,12.0
3,0,"35465 Robert Avenue Apt. 579\nPort Amberside, ...",F,76.313766,66.375720,Arts,76.620347,Comm&Mgmt,False,1,91.258958,Mkt&Fin,52.884747,NaN,False,2020-07-15,NaT,NaN
4,1,"9071 Max Views Suite 273\nSolisport, OR 45104",F,67.854861,72.825808,Arts,65.247689,Comm&Mgmt,True,2,81.867682,Mkt&Fin,59.950590,NaN,True,2020-03-06,NaT,NaN
5,3,"36399 Faulkner Trafficway Apt. 354\nMarcbury, ...",F,75.091458,65.394722,Science,68.207067,Comm&Mgmt,True,1,65.624899,Mkt&Fin,62.598471,NaN,False,2020-03-02,2020-10-05,12.0


`TVAE` also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, `TVAE` will not be able to set this value to 1000.

In [20]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,7,"802 Cline Valleys Apt. 856\nPort Derek, AZ 72941",M,76.782608,69.317137,Arts,70.0,Others,True,1,62.161332,Mkt&Fin,57.072286,NaN,False,2020-03-08,2020-09-04,NaN
1,11,"943 Villegas Summit\nPort Ashley, IA 58292",F,77.866785,66.472560,Science,70.0,Sci&Tech,False,0,63.044732,Mkt&Fin,51.188296,NaN,True,2020-03-04,2020-08-08,NaN
2,25,"5792 Cook Fords Apt. 034\nBrownport, WV 75373",F,73.818364,65.121089,Arts,70.0,Others,True,0,66.089684,Mkt&Fin,57.188872,NaN,False,2020-02-22,NaT,NaN
3,2,"938 Gordon Flat Suite 728\nSouth Daltonburgh, ...",F,73.611320,75.154360,Arts,70.0,Comm&Mgmt,False,0,63.214425,Mkt&Fin,63.944442,NaN,False,2020-03-04,NaT,NaN
4,16,"787 Gary Grove Suite 548\nJonesbury, MI 11394",F,73.632254,67.198328,Arts,70.0,Others,False,2,88.060749,Mkt&Fin,58.161713,23682.419271,False,2020-01-21,2020-07-25,NaN


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
types of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.